In [1]:
pip install wikipedia


  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=c9d1abdaca412f5875b1d1965e7903db9330b685d0570d3df7ec72c2374f3314
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import argparse
import lxml.etree
import wikipedia
import time

In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [4]:
import os
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from tqdm.notebook import tqdm
os.chdir('/gdrive/My Drive/NLP_Project_Programs/Retrofitting_ESA')
os.listdir()

['Graphs',
 'Datasets',
 'OtherFiles',
 'Evaluation_Benchmarks',
 'TOEFL Task.ipynb',
 'Wikipedia-RG65_10titles.xml',
 'Wikipedia-RG65_20titles.xml',
 'Wikipedia-MEN_5titles.xml',
 'Wikipedia-MEN_3titles.xml',
 'Eval_Dataset_String.ipynb',
 'Graph_Processing.ipynb',
 'Sentiment_Analysis.ipynb',
 'Word Similarity Tasks.ipynb',
 'explicit_semantic_anlaysis+retrofitting.ipynb',
 'titles_extract.ipynb']

In [0]:
def get_articles(max_no_articles, search, **kwargs):
    """ Retrieve articles from Wikipedia """
    wikipedia.set_rate_limiting(True) # be polite
    wikipedia.set_lang('EN')
    titles = wikipedia.search(search, results = max_no_articles)
    #if search is not None:
      #titles = wikipedia.search(search, results = max_no_articles)
    #else:
        #titles = wikipedia.random(pages = max_no_articles)
    #print(type(titles))
    articles = []
    for title in titles:
        #print("Title: {}".format(title))
        #page = wikipedia.page(title=title)
        #content = page.content
        #print(type(content))
        #article_no_words = len(content.split())
        #print("article contains {} words".format(article_no_words))
        articles.append(title)

    return articles

#Word Similarity

In [7]:
path_to_dataset = 'Evaluation_Benchmarks/Word_Similarity/WS353_Exact/EN-WS-353-ALL.txt'
path = 'WS_6titles.txt'
file = open(path,'w')

number_titles = 6
MEN = open(path_to_dataset,'r')
count = 0
for index,line in tqdm(enumerate(MEN)):
  #if count ==2: break
  count += 1
  line = line.strip().lower().split()
  word_pair = (line[0], line[1])
  titles_list1 = get_articles(number_titles,word_pair[0])
  titles_list2 = get_articles(number_titles,word_pair[1])
  file.write(word_pair[0] + "--" + "--".join(titles_list1)+"\n")
  file.write(word_pair[1] + "--" + "--".join(titles_list2)+"\n")

file.close()

In [8]:
path = 'WS_6titles.txt'
file1 = open(path,'r')
total_titles = []
count = 0
for index,line in tqdm(enumerate(file1)):
  #if count ==2: break
  count+=1
  line = line.strip().split('--')
  for title in line[1:]: 
    total_titles.append(title)

In [9]:
unique_total = set(total_titles)
print(len(total_titles))
print(len(unique_total))

4236
2630


In [10]:
Doubtless = []
for title in unique_total:
  if title.endswith('(disambiguation)'):
    print(title)
    continue
  else:
    Doubtless.append(title)  

Constellation (disambiguation)
Match (disambiguation)
Hundred (disambiguation)
Woodland (disambiguation)
Secret (disambiguation)
War (disambiguation)
Sign (disambiguation)
Volunteer (disambiguation)
Sex (disambiguation)
Egg (disambiguation)
Day (disambiguation)
Atmosphere (disambiguation)
Stupid (disambiguation)
Maradona (disambiguation)
Virtuoso (disambiguation)
Team (disambiguation)
Music (disambiguation)
Prominence (disambiguation)
Cat (disambiguation)
Rooster (disambiguation)
Oil (disambiguation)
Five by Five (disambiguation)
Boy (disambiguation)
Video (disambiguation)
News (disambiguation)
5 (disambiguation)
Mind (disambiguation)
Gem (disambiguation)
Sea (disambiguation)
Jaguar (disambiguation)
Carnivore (disambiguation)
Interest (disambiguation)
Star (disambiguation)
Love (disambiguation)
Bed (disambiguation)
Ear (disambiguation)
7 (disambiguation)
Tool (disambiguation)
Brandy (disambiguation)
Moon (disambiguation)
Midday (disambiguation)
Coffee (disambiguation)
Wood (disambiguat

In [11]:
print(len(Doubtless))

2556


In [0]:
exp_path = "ws353_6titles_1perline.txt"
obj = open(exp_path,'w')
for title in Doubtless:
  obj.write(title + "\n")
obj.close()


In [0]:
art = get_articles(10,'artificial intelligence')
print(art)

<class 'list'>
['Artificial intelligence', 'Artificial general intelligence', 'A.I. Artificial Intelligence', 'History of artificial intelligence', 'Glossary of artificial intelligence', 'Applications of artificial intelligence', 'Friendly artificial intelligence', 'Philosophy of artificial intelligence', 'Ethics of artificial intelligence', 'Distributed artificial intelligence']


In [0]:
a = ['aa','ss','ww']
b = "--".join(a)
print(b)

aa--ss--ww


#Sentiment Analysis

In [0]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
pth_1= 'Evaluation_Benchmarks/Sentiment_Analysis/Dataset_Exact/train.txt'


tr_obj = open(pth_1,'r')

count = 0
X_train_sentence = []
for index,line in tqdm(enumerate(tr_obj)):
  #if count ==20: break
  count += 1
  line = line.strip().split()
  temp = []
  for word in line[1:]:
    #print("word:", word)
    if word.isalpha() and word not in stop_words:
      #print("hahahaha")
      temp.append(word.lower())
  X_train_sentence.append(temp)
  
tr_obj.close()



In [0]:
def get_articles(search):
    """ Retrieve articles from Wikipedia """
    wikipedia.set_rate_limiting(True) # be polite
    wikipedia.set_lang('EN')
    titles_list = [wikipedia.search(search,results = 1)]
    #if search is not None:
      #titles = wikipedia.search(search, results = max_no_articles)
    #else:
        #titles = wikipedia.random(pages = max_no_articles)


    return articles

In [0]:
file = open(path_w,'w')
path_w = 'SA-Train_1title.txt'
number_titles = 1
count =0
wikipedia.set_rate_limiting(True) # be polite
wikipedia.set_lang('EN')
import time

for index,sentence in tqdm(enumerate(X_train_sentence)):
  
  #count+=1
  #if count == 3: break
  for word in sentence:
    #start = time.time()
    titles_list = wikipedia.search(word,results = 1)
    #print("took time:", time.time()-start)
    if len(titles_list) != 0:
      file.write(titles_list[0]+"\n")
    else: continue  
file.close()

In [0]:
print(index, word)

6919 seams


In [0]:
line = ['ppppp', 'aaa','ssss','dddd']
b = [word.isalpha() for word in line]
print(b) 

[True, True, True, True]


In [0]:
file1 = open(path_w,'r')
total_titles = []
count = 0
for index,line in tqdm(enumerate(file1)):
  #if count ==4: break
  count+=1
  line = line.strip()
  total_titles.append(line)
#print(total_titles)  

In [0]:
unique_total = set(total_titles)
print(len(total_titles))
print(len(unique_total))

62831
10576


In [0]:
Doubtless = []
for title in unique_total:
  if title.endswith('(disambiguation)'):
    print(title)
    continue
  else:
    Doubtless.append(title)  

Love (disambiguation)
Agent provocateur (disambiguation)
Either/Or (disambiguation)
Inflation (disambiguation)
Damn (disambiguation)
Imago (disambiguation)
Capitalization (disambiguation)
3 (disambiguation)
Purism (disambiguation)
Prominence (disambiguation)
Fistfight (disambiguation)
Water (disambiguation)
Crescendo (disambiguation)
Relay (disambiguation)
Divergence (disambiguation)
Relief (disambiguation)
Men (disambiguation)
Gobbler (disambiguation)
Result (disambiguation)
Complaint (disambiguation)
II (disambiguation)
Embarkation (disambiguation)
Etching (disambiguation)
4 (disambiguation)
Efficiency (disambiguation)
Containment (disambiguation)
Dead (disambiguation)
Bite (disambiguation)
Suffocation (disambiguation)
Secret (disambiguation)
Appeal (disambiguation)
Interest (disambiguation)
Eruption (disambiguation)
Video (disambiguation)
One (disambiguation)
Arrogance (disambiguation)
Climbing (disambiguation)
Intersection (disambiguation)
Like (disambiguation)
ADD (disambiguation)

In [0]:
print(len(Doubtless))

10533


In [0]:
exp_path = "SA-train_1title_wiki.txt"
obj = open(exp_path,'w')
for title in Doubtless:
  obj.write(title + "\n")
obj.close()
